In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### about shape

In [4]:
x = torch.tensor(1)
y = torch.tensor([1])
z = torch.tensor([[1]])
print(x.shape)
print(y.shape)
print(z.shape)

torch.Size([])
torch.Size([1])
torch.Size([1, 1])


### torch.cat(tensors, dim=0)
All tensors must have the same shape

In [9]:
x = torch.tensor(1)
y = torch.tensor([1])
z = torch.tensor([[1]])
# print(torch.cat([x, x])) Error
print(torch.cat([y, y]))
print(torch.cat([z, z]))

tensor([1, 1])
tensor([[1],
        [1]])


### about view and flatten

 ### torch.max

In [5]:
a = torch.randn(4, 4)
a

tensor([[ 0.1758, -1.7297,  0.6679, -0.3804],
        [-0.7364, -0.5691, -0.0367,  0.2619],
        [-1.4519,  0.6855,  1.0648,  1.0596],
        [-0.9962,  0.7628,  0.3344, -0.1493]])

In [6]:
torch.max(a, dim=1)

torch.return_types.max(
values=tensor([0.6679, 0.2619, 1.0648, 0.7628]),
indices=tensor([2, 3, 2, 1]))

### torch.gather

In [4]:
class QNetwork(nn.Module):
    def __init__(self):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 4)
    
    def forward(self, x):
        h = F.relu(self.fc1(x))
        out = self.fc2(h)
        return out

model = QNetwork()
x = torch.randn(size=(10, 128))
out = model(x)
print(out)

tensor([[ 0.2624,  0.5077, -0.0924,  0.2940],
        [ 0.1414,  0.3631,  0.3091,  0.2611],
        [ 0.0592, -0.1569, -0.1662, -0.0604],
        [ 0.1527,  0.6744,  0.1132, -0.1710],
        [-0.0991,  0.2439, -0.2503, -0.2905],
        [-0.0063, -0.2551, -0.2488, -0.0402],
        [ 0.1821, -0.0663, -0.0793,  0.1456],
        [ 0.4182,  0.4052, -0.4260, -0.4240],
        [ 0.3985,  0.1850,  0.2596,  0.0417],
        [ 0.3793,  0.2247,  0.3346, -0.0804]], grad_fn=<AddmmBackward0>)


In [5]:
indexs = torch.tensor([[0], [1], [2], [3], [0], [1], [2], [3], [1], [2]])
out.gather(1, indexs)

tensor([[ 0.2624],
        [ 0.3631],
        [-0.1662],
        [-0.1710],
        [-0.0991],
        [-0.2551],
        [-0.0793],
        [-0.4240],
        [ 0.1850],
        [ 0.3346]], grad_fn=<GatherBackward0>)

### torch.index_select

In [14]:
x = torch.randn(3, 4)
print("x = ")
print(x)
indices = torch.tensor([0, 2])
print("dimention 0:")
print(torch.index_select(x, 0, indices))
print("dimention 1:")
print(torch.index_select(x, 1, indices))

x = 
tensor([[ 1.2081,  0.2444,  2.0005, -0.3794],
        [-0.3914,  1.0535,  0.7878,  1.6014],
        [-1.2875, -0.2883,  1.6580, -0.9259]])
dimention 0:
tensor([[ 1.2081,  0.2444,  2.0005, -0.3794],
        [-1.2875, -0.2883,  1.6580, -0.9259]])
dimention 1:
tensor([[ 1.2081,  2.0005],
        [-0.3914,  0.7878],
        [-1.2875,  1.6580]])


### SoftmaxとCategorical

In [15]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(5, 32)
        self.fc2 = nn.Linear(32, 4)
    
    def forward(self, x):
        h = self.fc1(x)
        out = self.fc2(h)
        return out

In [16]:
x = torch.randn(size=(10, 5))
x

tensor([[-0.3641,  0.0067,  1.6599, -0.5478, -1.1946],
        [ 1.3941,  0.5103, -1.2828,  0.1066, -1.3573],
        [ 1.1782, -0.9006, -0.9926, -1.2791, -0.6572],
        [-1.4405,  0.2087, -0.2380, -1.4080,  0.6785],
        [-0.7524,  0.4348,  0.8985, -0.9169, -1.3506],
        [ 0.3669, -1.1351,  0.3406, -0.8650, -0.6660],
        [ 0.5459,  1.0183, -0.5866,  0.0033,  0.5139],
        [-0.2817,  0.4276,  0.0028, -0.9663, -0.0124],
        [-0.5379,  1.2812, -1.9080,  0.8128,  1.3401],
        [ 0.9822,  0.7250, -1.9718,  1.0767, -0.5611]])

In [19]:
model = Policy()
logits = model(x)
print(logits)

tensor([[ 0.1457,  0.7777, -0.0076, -0.1573],
        [ 0.2245, -0.0301, -1.0757, -0.3668],
        [-0.5540,  0.2869, -1.1656, -0.2759],
        [-0.1492, -0.4192,  0.0541, -0.3104],
        [ 0.0324,  0.5656, -0.0331, -0.2971],
        [-0.2354,  0.5225, -0.7100, -0.1889],
        [ 0.3637, -0.6060, -0.3224, -0.2035],
        [-0.0595, -0.0761, -0.1837, -0.2452],
        [ 0.9091, -1.6545, -0.2811, -0.3805],
        [ 0.7730, -0.8078, -1.0308, -0.4151]], grad_fn=<AddmmBackward0>)


In [30]:
softmax = F.softmax(logits, dim=1)
print(softmax)

tensor([[0.2233, 0.4202, 0.1916, 0.1650],
        [0.3844, 0.2980, 0.1048, 0.2128],
        [0.1930, 0.4475, 0.1047, 0.2549],
        [0.2604, 0.1988, 0.3191, 0.2217],
        [0.2293, 0.3909, 0.2148, 0.1650],
        [0.2082, 0.4442, 0.1295, 0.2181],
        [0.4082, 0.1548, 0.2055, 0.2315],
        [0.2705, 0.2660, 0.2389, 0.2246],
        [0.6036, 0.0465, 0.1836, 0.1662],
        [0.5969, 0.1229, 0.0983, 0.1819]], grad_fn=<SoftmaxBackward0>)
tensor([[-1.4992, -0.8671, -1.6525, -1.8021],
        [-0.9560, -1.2106, -2.2562, -1.5473],
        [-1.6451, -0.8042, -2.2567, -1.3670],
        [-1.3454, -1.6155, -1.1422, -1.5066],
        [-1.4725, -0.9393, -1.5380, -1.8021],
        [-1.5694, -0.8114, -2.0440, -1.5229],
        [-0.8960, -1.8657, -1.5821, -1.4632],
        [-1.3076, -1.3242, -1.4318, -1.4933],
        [-0.5048, -3.0683, -1.6949, -1.7943],
        [-0.5160, -2.0968, -2.3198, -1.7041]], grad_fn=<LogBackward0>)


In [29]:
m = torch.distributions.categorical.Categorical(softmax)
m.sample()

tensor([0, 0, 0, 2, 1, 1, 0, 2, 0, 0])